In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!nvidia-smi

Wed Apr  8 01:55:43 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
import os
import cv2
from time import time
import sys
#sys.path.append('../plategen')
sys.path.append('drive/My Drive/wcp/crnn')
sys.path.append('drive/My Drive/wcp/plategen')
from plategen import TaiwanLicensePlateGenerator
    
now = time()

path = './test'
if not os.path.isdir(path):
    os.mkdir(path)
g = TaiwanLicensePlateGenerator('drive/My Drive/wcp/plategen/char_black', 'drive/My Drive/wcp/plategen/char_s/-.jpg',
                                'drive/My Drive/wcp/plategen/border/car_plate_border.png', True)
#g = TaiwanLicensePlateGenerator('../plategen/char_black', '../plategen/char_s/-.jpg',
#                                '../plategen/border/car_plate_border.png', True)
plates, labels = g.gen_plate(2800, 'perspective')
for plate, label in zip(plates, labels):
    cv2.imwrite(path+'/'+label+'.jpg', plate)
print(f'output to {path} for overall {time()-now:.2f} secs')

34 chars ['0', '1', '2', '3', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '-'] is loadeded
2,800 car plates is generated
output to ./test for overall 15.05 secs


In [0]:
from zipfile import ZipFile
z = ZipFile('/content/drive/My Drive/wcp/crnn/test.zip')
z.extractall('')

In [0]:
import sys
sys.path.append('drive/My Drive/wcp/crnn')
sys.path.append('drive/My Drive/wcp/plategen')
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adadelta
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
#from Image_Generator import TextImageGenerator
from Model_tf2 import get_Model
#from Model import get_Model
from parameter import *
from imagegen import ImageGenerator
K.set_learning_phase(0)

# # Model description and training

model = get_Model(training=True)

try:
    model.load_weights('drive/My Drive/wcp/crnn/model/wcp_0407_random-90-12.465.hdf5')
    #model.load_weights('drive/My Drive/LPR/crnn/model/abc.hdf5')
    print("...Previous weight data...")
except:
    print("...New weight data...")
    pass

train_size = 70000
train_batch_size = 192
tiger_train = ImageGenerator(None, img_w, img_h, train_batch_size, downsample_factor,
                             'drive/My Drive/wcp/plategen/char_black',
                             'drive/My Drive/wcp/plategen/char_s/-.jpg',
                             'drive/My Drive/wcp/plategen/border/car_plate_border.png',
                             max_text_len)

valid_batch_size = 192
tiger_val = ImageGenerator('test/', img_w, img_h, valid_batch_size, downsample_factor, max_text_len=max_text_len)
tiger_val.build_data()

ada = Adadelta()

early_stop = EarlyStopping(monitor='loss', min_delta=0.001, patience=4, mode='min', verbose=1)
checkpoint = ModelCheckpoint(filepath='drive/My Drive/wcp/crnn/model/wcp_0407_random+ep90-{epoch:02d}-{val_loss:.3f}.hdf5',
                             monitor='loss', verbose=1, mode='min', period=5)
#checkpoint = ModelCheckpoint(filepath='model/wcp_0407-{epoch:02d}-{val_loss:.3f}.hdf5', monitor='loss', verbose=1, mode='min', period=5)
# the loss calc occurs elsewhere, so use a dummy lambda func for the loss
model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer=ada)

# captures output of softmax so we can decode the output during visualization
model.fit_generator(generator=tiger_train.next_batch(),
                    #samples_per_epoch=int(train_size/train_batch_size),
                    steps_per_epoch=int(train_size / train_batch_size),
                    epochs=600,
                    callbacks=[checkpoint],
                    validation_data=tiger_val.next_batch(),
                    validation_steps=int(tiger_val.n / valid_batch_size))

...Previous weight data...
2800  Image Loading start...
True
2800  Image Loading finish...
Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/600
364/364 [==============================] - 397s 1s/step - loss: 12.1977 - val_loss: 12.1757
Epoch 2/600
364/364 [==============================] - 394s 1s/step - loss: 11.9630 - val_loss: 11.9112
Epoch 3/600
364/364 [==============================] - 398s 1s/step - loss: 11.7409 - val_loss: 11.7476
Epoch 4/600
364/364 [==============================] - 395s 1s/step - loss: 11.5216 - val_loss: 11.5198
Epoch 5/600
364/364 [==============================] - ETA: 0s - loss: 11.3363
Epoch 00005: saving model to drive/My Drive/wcp/crnn/model/wcp_0407_random+ep90-05-11.308.hdf5
364/364 [==============================] - 400s 1s/step - loss: 11.3363 - val_loss: 11.3084
Epoch 6/600
364/364 [==============================] - 402s 1s/step - loss: 11.1052 - val_loss: 11.0080
Epoch 7/600
364/364 [==========================